# **AI Agents and Assistants**

Technology Used:

![Python](https://img.shields.io/badge/python-v3.7+-blue.svg)
![Jupyter](https://img.shields.io/badge/jupyter-v1.0+-blue.svg)
![CrewAI](https://img.shields.io/badge/crewai-v0.0.1-blue.svg)
![DuckDuckGo](https://img.shields.io/badge/duckduckgo_search-v0.0.1-blue.svg)
![OpenAI](https://img.shields.io/badge/openai-v0.0.1-blue.svg)
![Google Colab](https://img.shields.io/badge/google_colab-v1.0+-blue.svg)

## **1. AI Agents**

<img src="https://mapxp.app/MBA742/AI_agent.png" width="900"/>

<br><br>

--------
<br><br>

<img src="https://mapxp.app/MBA742/AI_agent_email.png" width="900"/>

<br><br>

--------
<br><br>


<img src="https://mapxp.app/MBA742/AI_agent_productivity.png" width="900"/>

<br><br>

--------
<br><br>

<img src="https://mapxp.app/MBA742/AI_agent_humans.png" width="900"/>

<br><br>

--------
<br><br>

<img src="https://mapxp.app/MBA742/AI_agent_chatbots.png" width="900"/>

<br><br>

--------
<br><br>

<img src="https://mapxp.app/MBA742/AI_agent_platforms.png" width="900"/>

<br><br>

## **2. Building an AI Agent with CrewAI**

https://www.crewai.com/open-source

**Objective:** Automate writing a LinkedIn post on a topic of interest.  

**Requirements:**
* Contemporary Data
* Relevant Insights
* Text in the style and format of a LinkedIn post

**Approach:**
1. Collect data on topic of interest
2. Synthetsize collected data
3. Extract key insights and messages
4. Create a post suitable for LinkedIn

## **2.1 Install and Import Libraries**

In [ ]:
# --- Install missing Libraries and Packages ---
!sudo apt-get update
!pip install --q crewai
!pip install --q 'crewai[tools]'
!pip install -U duckduckgo_search

In [ ]:
# --- Import all Libraries and Packages ---
import os
from google.colab import userdata

from crewai import Agent, Task, Process, Crew, LLM
from crewai_tools import ScrapeWebsiteTool
from crewai.tools import tool

from duckduckgo_search import DDGS
import re

## **2.2 What is your Agent Researching?**

In [ ]:
# --- Define Research Topic ---
research_topic = "the future of a computer science degree in the age of AI"

## **2.3 Select LLMs and other Tools**

* Need genAI for insight extraction and writing
* Need to find content on the web
* Need to scrape (download) content from the web

In [ ]:
# --- Set API Key ---
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# --- Configure genAI Models ---
llm = LLM(
    model="gpt-4o",
    # temperature=0.7,
    # timeout=120,
    # max_tokens=4000,
    # top_p=0.9,
    # frequency_penalty=0.1,
    # presence_penalty=0.1,
    # seed=42
)

llm_reason = LLM(
    model="o1-mini"
)

In [ ]:
# --- Set-up Tools ---

# Tool 1 : Search for relevant Webpages
@tool
def duckduckgo_URLs(search_query: str):
    """Wrapper for crewai around duckduckgo_search: Searches the web using DuckDuckGo and returns a summary of findings."""
    results = DDGS().text(search_query, max_results=5)
    urls = []
    for result in results:
      if 'href' in result:
        url = result['href']
        urls.append(url)
    return urls

# Tool 2 : CrewAI internal tool: Scrapes webpages
read_webpage = ScrapeWebsiteTool()

## **2.4 Design your Agents**

In [ ]:
# --- Define Agents ---
researcher = Agent(
    role="Senior Researcher",
    goal=f"Find the most relevant webpages for {research_topic} using DuckDuckGo.",
    backstory="An expert in online research, capable of identifying the most relevant webpages for a given topic.",
    llm=llm,
    tools=[duckduckgo_URLs]
)

analyst = Agent(
    role="Senior Analyst",
    goal=f"Find the most relevant content on {research_topic} on a webpage.",
    backstory="An expert in extracting relevant information from webpages. \
              Documents and includes all relevant sources (including URL) to each piece of information",
    llm=llm,
    #llm=llm_reason,
    tools=[read_webpage]
)

writer = Agent(
    role="Experienced Journalist",
    goal="Write a well-structured and engaging report based on the research findings. \
          Make sure to cite relevant sources (including URL) for all claims and statements.",
    backstory="A skilled writer specializing in translating complex information into clear and understandable language.",
    llm=llm
    #llm=llm_reason
)

blogger = Agent(
    role="Expert Blogger",
    goal="Write a compact and compelling LinkedIn post in typical languge.\
          Use emojis and include relevant hashtags #. \
          Refer to people, firms and organizations with mentions @ .",
    backstory="A social media expert and successful blogger who has thousands of followers on LinkedIn \
              and knows how to write engaging posts.",
    llm=llm
)

## **2.5 Define Tasks for Agents**

In [ ]:
# --- Define Tasks ---
research_task = Task(
    description=f"Use DuckDuckGo to find webpages with latest trends and news from 2024 and 2025 on {research_topic}.",
    expected_output="A list of relevant URLs",
    agent=researcher
)

extraction_task = Task(
    description=f"Visit each webpage and extract the most relevant insights on {research_topic}",
    expected_output=f"A well-structured narrative of key insights, each citing supporting sources.",
    agent=analyst,
    context=[research_task]
)

writing_task = Task(
    description="Transform the insights into a punchy executive summary with a call to action.",
    expected_output="A well-structured and engaging executive summary with clear action points. \
                    All sources at the bottom under *References*, including their URL.",
    agent=writer,
    context=[extraction_task]
)

post_task = Task(
    description="Translate the executive summary into an engaging and thought-provoking LinkedIn post.",
    expected_output="A polished LinkedIn post with no more than 200 words that provokes comments and likes.",
    agent=blogger,
    context=[writing_task]
)

## **2.6 Define a Workflow**

* Involved Agents
* Tasks
* Process

In [ ]:
# --- Define and Run Crew Process ---
LinkedIn_crew = Crew(
    agents=[researcher, analyst, writer, blogger],
    tasks=[research_task, extraction_task, writing_task, post_task],
    process=Process.sequential,
    verbose=True # set to false if you don't want to see the crew work
)

## **2.7 Launch your Agents**

In [ ]:
# ---  Put your AI Agent Crew to work ---
results = LinkedIn_crew.kickoff()
print("✅ Your crew has finished their work.")

## **2.8 Examine Results**

In [ ]:
# ---  Check out your AI Crew's Work ---
print(results)

In [ ]:
# ---  See how we got there: Extract all task outputs ---
for task in results.tasks_output:
    print(f"\n🔹 Task: {task.description}")
    print(f"👤 Performed by: {task.agent}")
    print(f"📌 Expected Output: {task.expected_output}")
    print(f"📜 Generated Output:\n\n{task.raw}\n")

## **3. An Agentic World**

<img src="https://mapxp.app/MBA742/AI_agent_lives.png" width="900"/>

<br><br>

## 4. AI Cloaking: Techniques and Implications

**AI Cloaking** refers to strategies designed to make AI-generated content indistinguishable from human-generated content. In the context of AI agents and content creation, cloaking can help:
- **Enhance Credibility:**
  - By refining language and style, AI-generated text appears more natural, reducing the risk of detection as machine-produced.
- **Improve User Engagement:**
  - Cloaked content better resonates with audiences who expect authentic, human-like communication.
- **Mitigate Ethical and Privacy Concerns:**
  - In some contexts, it is beneficial to hide the origin of the content to protect proprietary processes or comply with privacy guidelines.

### Techniques for AI Cloaking
1. **Style Mimicry:**
   - Adjust model parameters (such as temperature, frequency, and presence penalties) to fine-tune the tone and style of the generated text.
   - Use human feedback loops to refine and "humanize" the language.

2. **Post-Processing Filters:**
   - Apply linguistic filters and natural language processing (NLP) techniques to remove common artifacts of AI generation (e.g., repetitive phrasing or overly formal language).
   - Introduce variability in sentence structure and word choice to mimic human idiosyncrasies.

3. **Contextual Embedding:**
   - Incorporate contextual details that are typical of human authorship, such as personal anecdotes or references to current events, to enrich the narrative.

4. **Hybrid Generation Approaches:**
   - Combine human editing with AI-generated drafts to create a final output that leverages the strengths of both.
   - Use AI for the heavy lifting of research and synthesis, then have human oversight for style and tone adjustments.

### Implications of AI Cloaking
- **For Business:**
  - Enables the seamless integration of AI-generated content into corporate communications, marketing, and strategic messaging.
- **For Research:**
  - Provides a tool for generating high-quality, indistinguishable content that can be used in experiments or case studies.
- **For Ethics:**
  - Raises important questions about transparency and the ethical use of AI, making it critical to consider when and how cloaking should be employed.

<img src="https://mapxp.app/MBA742/AI_agent_cloaking.png" width="900"/>


In [ ]:
# Export this notebook to markdown

from google.colab import drive
from google.colab import files

drive.mount('/content/drive')

# Convert to markdown: make sure that your path and filename are correct
!jupyter nbconvert --to markdown "/content/drive/MyDrive/488/Class11/488_2025_Class11.ipynb"

# download file: make sure that your path and filename are correct and consistent with those above
files.download("/content/drive/MyDrive/488/Class11/488_2025_Class11.md")